In [ ]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir

import os
import json
import _jsonnet

import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import display, Javascript, HTML
from traitlets import Unicode, Bool, validate, TraitError
from ipywidgets import (DOMWidget, register, interact, interactive,  interact_manual,
                        IntSlider, IntText, FloatText, Text, BoundedIntText,
                        BoundedFloatText, Box, HBox, VBox, fixed, Dropdown, Label,
                        Layout, Tab, Accordion, Button, ToggleButton, ToggleButtons, Checkbox)

In [ ]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
k12ai_print(cv_json)

In [ ]:
k12cv_conf = json.loads(cv_json)

In [ ]:
out = widgets.Output()

def k12widget(method):
    def _widget(self, *args, **kwargs):
        root, observes, cb = method(self, *args, **kwargs)
        def _on_value_change(change, cb):
            with out:
                clear_output()
                print(change)
            if cb:
                cb(change)
                
        for per in observes:
            per.observe(lambda change, cb = cb: _on_value_change(change, cb), 'value')
        return root
    return _widget

In [ ]:
class K12Context():
    def __init__(self, defaults=None):
        self.wids_map = {}
        self.defaults = defaults
        self.vlo = Layout(
            width='auto',
            align_items='stretch',
            justify_content='flex-start',
            margin='3px 3px 3px 3px',
            # border='solid 1px red',
        )
        self.hlo = Layout(
            width='100%',
            flex_flow='row wrap',
            align_items='stretch',
            justify_content='flex-start',
            margin='3px 0px 3px 0px',
            # border='solid 1px blue',
        )
        self.style = {
            # 'description_width': 'initial',
            'description_width': '120px',
        }

In [ ]:
@k12widget
def Bool(self, wid, *args, **kwargs):
    self.wids_map[wid] = Checkbox(*args, **kwargs)
    def _value_change(change):
        print("Checkbox here")
    return self.wids_map[wid], [self.wids_map[wid]], _value_change
   
K12Context.Bool = Bool    

In [ ]:
@k12widget
def Int(self, wid, *args, **kwargs):
    self.wids_map[wid] = BoundedIntText(*args, **kwargs)
    def _value_change(change):
        print("BoundedIntText here")
    return self.wids_map[wid], [self.wids_map[wid]], _value_change

K12Context.Int = Int

In [ ]:
@k12widget
def Float(self, wid, *args, **kwargs):
    self.wids_map[wid] = BoundedFloatText(*args, **kwargs)
    def _value_change(change):
        print("BoundedFloatText here")
    return self.wids_map[wid], [self.wids_map[wid]], _value_change

K12Context.Float = Float

In [ ]:
@k12widget
def String(self, wid, *args, **kwargs):
    self.wids_map[wid] = Text(*args, **kwargs)
    return self.wids_map[wid], [self.wids_map[wid]], None

K12Context.String = String

In [ ]:
@k12widget
def Array(self, wid, *args, **kwargs):
    self.wids_map[wid] = Text(*args, **kwargs)
    def _value_change(change):
        print("IntArray here")
        print(json.loads(change['new']))
    return self.wids_map[wid], [self.wids_map[wid]], _value_change

K12Context.Array = Array

In [ ]:
@k12widget
def StringEnum(self, wid, *args, **kwargs):
    self.wids_map[wid] = Dropdown(*args, **kwargs)
    def _value_change(change):
        print("StringEnum here")
    return self.wids_map[wid], [self.wids_map[wid]], _value_change

K12Context.StringEnum = StringEnum

In [ ]:
@k12widget
def BoolTrigger(self, wid, *args, **kwargs):
    parent_box = VBox(layout = self.vlo)
    parent_box.trigger_box = VBox(layout = self.vlo)
    
    wdg = Checkbox(*args, **kwargs)
    wdg.parent_box = parent_box
    def _switch_widget(wdg, value):
        if value:
            trigger_box = wdg.parent_box.trigger_box
            wdg.parent_box.children = [wdg] + list(trigger_box.children)
        else:
            wdg.parent_box.children = [wdg]

    def _value_change(change):
        print("BoolTrigger here")
        wdg = change['owner']
        val = change['new']
        _switch_widget(wdg, val)
    _switch_widget(wdg, wdg.value)
    return parent_box, [wdg], _value_change

K12Context.BoolTrigger = BoolTrigger

In [ ]:
@k12widget
def StringEnumTrigger(self, wid, *args, **kwargs):
    parent_box = VBox(layout = self.vlo)
    wdg = Dropdown(*args, **kwargs)
    wdg.parent_box = parent_box
    parent_box.trigger_box = {
     value: VBox(layout = self.vlo) for _, value in wdg.options
    }

    def _switch_widget(wdg, value):
        trigger_box = wdg.parent_box.trigger_box[value]
        wdg.parent_box.children = [wdg] + list(trigger_box.children)
        # wdg.parent_box.children = [wdg, wdg.parent_box.trigger_box[value]]

    def _value_change(change):
        print("StringEnumTrigger here")
        wdg = change['owner']
        val = change['new']
        _switch_widget(wdg, val)
    _switch_widget(wdg, wdg.value)
    return parent_box, [wdg], _value_change

K12Context.StringEnumTrigger = StringEnumTrigger

In [ ]:
@k12widget
def StringEnumGroupTrigger(self, wid, options, groups):
    parent_box = VBox(layout = self.vlo)
    parent_box.trigger_box = {}
    options_hbox = HBox(layout = self.hlo)
    dynamic_hbox = HBox(layout = self.hlo)
    observes = []
    for name, real in options:
        dpd = Dropdown(options=groups,
                       description=name,
                       style = {'description_width': '150px'})
        dpd.name = real
        dpd.wid = wid + '.' + real 
        dpd.parent_box = parent_box
        dpd.parent_box.trigger_box[real] = VBox(layout = self.vlo)
        observes.append(dpd)            
    options_hbox.children = tuple(observes)
    parent_box.children = (options_hbox, dynamic_hbox)
    def _value_change(change):
        print("StringEnumGroupTrigger here")
        wdg = change['owner']
        val = change['new']
        parent_box = wdg.parent_box
        dynamic_box = parent_box.children[1]
        trigger_box = parent_box.trigger_box[wdg.name]
        if val == 'none':
            if trigger_box in dynamic_box.children:
                dynamic_box.children = [ w for w in dynamic_box.children if id(w) != id(trigger_box)]
        else:
            if trigger_box not in dynamic_box.children:
                dynamic_box.children = list(dynamic_box.children) + [trigger_box]
    return parent_box, observes, _value_change

K12Context.StringEnumGroupTrigger = StringEnumGroupTrigger

In [ ]:
@k12widget
def Navigation(self, wid, *args, **kwargs):
    parent_box = VBox(layout = self.vlo)
    wdg = ToggleButtons(*args, **kwargs)
    wdg.parent_box = parent_box
    trigger_box = {}
    for name in wdg.options:
        trigger_box[name] = VBox(layout = self.vlo)
    wdg.parent_box.trigger_box = trigger_box

    def _switch_widget(wdg, value):
        parent_box = wdg.parent_box
        trigger_box = parent_box.trigger_box[value]
        parent_box.children = [wdg, trigger_box]

    def _value_change(change):
        print("ObjectEnumTrigger here")
        wdg = change['owner']
        val = change['new']
        _switch_widget(wdg, val)
    _switch_widget(wdg, wdg.value)
    return parent_box, [wdg], _value_change
        
K12Context.Navigation = Navigation

In [ ]:
box_layout = Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='stretch',
    justify_content='flex-start',
    width='100%'
)

# this way: slow
# box_layout = {
#     'display': 'flex',
#     'flex_flow': 'row wrap',
#     'align_items': 'stretch',
#     'justify_content': 'flex-start',
#     'width': '100%'
# }

tab_layout = Layout(
    display='flex',
    width='100%',
)

accordion_layout = Layout(
    display='inline-flex',
    # justify_content='flex-start',
    # align_items='stretch',
    width='99%'
)

vlo = Layout(
    width='auto',
    margin='3px 3px 3px 3px',
    # border='solid 1px red',
)

hlo = Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='stretch',
    justify_content='flex-start',
    width='100%',
    margin='3px 0px 3px 0px',
    # border='solid 1px blue',
)

gstyle = {
    'description_width': 'initial',
}

_basic_types = ['int', 'float', 'bool', 'string', 'string-enum']

k12Context = K12Context()

In [ ]:
def _widget_add_child(widget, wdgs):
    if not isinstance(wdgs, list):
        wdgs = [wdgs]
    for child in wdgs:
        widget.children = list(widget.children) + [child]
    return widget

In [ ]:
lan = 'en'

BASIC = 0
COMPLEX = 1
    
def _parse_config(widget, config):
    __id_ = config.get('_id_', None) or ''
    _name = config.get('name', None)
    _type = config.get('type', None)
    _objs = config.get('objs', None) or {}
    
    if _type == 'page':
        wdg = Tab(layout = tab_layout)
        for obj in _objs:
            _parse_config(wdg, obj)
        return _widget_add_child(widget, wdg)
    
    elif _type == 'tab':
        if not isinstance(widget, Tab):
            raise RuntimeError('Configure Error')
        widget.set_title(len(widget.children), _name[lan])
        wdg = VBox(layout = vlo)
        for obj in _objs:
            _parse_config(wdg, obj)
        return _widget_add_child(widget, wdg)
    
    elif _type == 'accordion':
        for wdg in widget.children:
            if isinstance(wdg, Accordion):
                accord = wdg
                break
        else:
            accord = Accordion(layout = accordion_layout)
            _widget_add_child(widget, accord)
        accord.set_title(len(accord.children), _name[lan])
        
        wdg = VBox(layout = vlo)
        for obj in _objs:
            _parse_config(wdg, obj)
        return _widget_add_child(accord, wdg)
    
    elif _type == 'navigation':
        default = config.get('default', 'none')
        options = []
        for obj in _objs:
            if obj.get('name', None):
                options.append(obj['name'][lan])
        wdg = k12Context.Navigation(
            __id_,
            options = options,
            description = _name[lan])
        for obj in _objs:
            if obj.get('name', None):
                _parse_config(wdg.trigger_box[obj['name'][lan]], obj)                    
        return _widget_add_child(widget, wdg)
    
    elif _type == 'object':
        if _name:
            wdg = widgets.HTML(value = f"<b><font color='black'>{_name[lan]} :</b>")
            _widget_add_child(widget, wdg)
        for obj in _objs:
            _parse_config(widget, obj)
        return widget
    
    elif _type == 'HV':
        wdg = VBox(
            [HBox(layout = hlo), HBox(layout = hlo)],
            layout = vlo,
        )
        for obj in _objs:
            if obj.get('type') in _basic_types:
                _parse_config(wdg.children[0], obj)
            else:
                _parse_config(wdg.children[1], obj)
        return _widget_add_child(widget, wdg)
    
    elif _type == 'bool':
        default = config.get('default', False)
        wdg = k12Context.Bool(
            __id_,
            description = _name[lan],
            value = default,
         )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'int':
        default = config.get('default', 0)
        min = config.get('min', 0)
        max = config.get('max', 100)
        width = config.get('width', 200)
        wdg = k12Context.Int(
            __id_,
            description = _name[lan],
            value = default,
            min = min,
            max = max,
            step = 1,
            # layout = Layout(width = '%dpx' % width),
        )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'float':
        default = config.get('default', 0.0)
        min = config.get('min', 0.0)
        max = config.get('max', 100.0)
        width = config.get('width', 200)
        wdg = k12Context.Float(
            __id_,
            description = _name[lan],
            value = default,
            min = min,
            max = max,
            step = 1.0,
            # layout = Layout(width = '%dpx' % width),
        )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'string':
        default = config.get('default', 'none')
        width = config.get('width', 200)
        wdg = k12Context.String(
            __id_,
            description = _name[lan],
            value = default,
            # layout = Layout(width = '%dpx' % width),
        )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'int-array' or _type == 'float-array':
        default = config.get('default', '[]')
        wdg = k12Context.Array(
            __id_,
            description = _name[lan],
            value = json.dumps(default),
        )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'string-enum':
        default = config.get('default', 'None')
        width = config.get('width', 200)
        options = []
        for obj in _objs:
            options.append((obj['name'][lan], obj['value']))
        wdg = k12Context.StringEnum(
            __id_,
            options = options,
            value = default,
            description = _name[lan],
            # layout = Layout(width = '%dpx' % width),
        )
        return _widget_add_child(widget, wdg)
    
    elif _type == 'bool-trigger':
        default = config.get('default', False)
        wdg = k12Context.BoolTrigger(
            __id_,
            value = default,
            description = _name[lan])
        for obj in _objs:
            _parse_config(wdg.trigger_box, obj['trigger'])
        return _widget_add_child(widget, wdg)
    
    elif _type == 'string-enum-trigger':
        default = config.get('default', 'none')
        width = config.get('width', 200)
        options = []
        for obj in _objs:
            options.append((obj['name'][lan], obj['value']))
        wdg = k12Context.StringEnumTrigger(
            __id_,
            options = options,
            value = default,
            description = _name[lan],
            # layout = Layout(width = '%dpx' % width),
        )
        for obj in _objs:
            _parse_config(wdg.trigger_box[obj['value']], obj['trigger'])                    
        return _widget_add_child(widget, wdg)
    
    elif _type == 'string-enum-array-trigger':
        raise RuntimeError('not impl yet')
        
    elif _type == 'string-enum-group-trigger':
        options = []
        groups = []
        for obj in _objs:
            options.append((obj['name'][lan], obj['value']))
        for grp in config.get('groups'):
            groups.append((grp['name'][lan], grp['value']))
        wdg = k12Context.StringEnumGroupTrigger(
            __id_,
            options,
            groups,
        )
        for obj in _objs:
            _parse_config(wdg.trigger_box[obj['value']], obj['trigger'])                    
            
        description = widgets.HTML(value = f"<b><font color='black'>{_name[lan]} :</b>")
        return _widget_add_child(widget, [description, wdg])
    else:
        for obj in _objs:
            _parse_config(widget, obj)

page = Box(layout=Layout(width='100%'))
_parse_config(page, k12cv_conf)
display(page, out)